In [11]:
# !pip install openwakeword
# !pip install speechbrain
# !pip install datasets
# !pip install scipy matplotlib

In [12]:
import os
import collections
import numpy as np
from numpy.lib.format import open_memmap
from pathlib import Path

from onnxruntime.transformers.shape_infer_helper import file_path
from tqdm import tqdm
import openwakeword
# import openwakeword.data
import openwakeword.utils
import openwakeword.metrics


import scipy
import datasets
import matplotlib.pyplot as plt
import torch
from torch import nn
import IPython.display as ipd

import datasets
import os
import numpy as np
import scipy.io.wavfile
from tqdm import tqdm

import urllib.request


import warnings

warnings.filterwarnings("ignore")

In [13]:


ds = datasets.load_dataset(
    "speech_commands",
    "v0.02",
    split="test",
    streaming=True
)
ds_iter = iter(ds)

os.makedirs("./data/speech_commands_test", exist_ok=True)
limit = 4890

for i in tqdm(range(limit)):
    output_file = f"./data/speech_commands_test/{i:05d}.wav"

    if os.path.exists(output_file):
        continue

    example = next(ds_iter)
    wav_data = (example["audio"]["array"] * 32767).astype(np.int16) # Convert to 16-bit PCM Format
    scipy.io.wavfile.write(output_file, 16000, wav_data)

100%|██████████| 4890/4890 [00:00<00:00, 16187.49it/s]


# ⇲ Compute Audio Embeddings

Create audio pre-processing object to get openwakeword audio embeddings

In [14]:
model_dir = "./resources/models"
os.makedirs(model_dir, exist_ok=True)

List Model and URL corresponding

In [15]:
models = {
    "embedding_model.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx",
    "embedding_model.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite",
    "melspectrogram.onnx": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx",
    "melspectrogram.tflite": "https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite"
}

In [16]:
for filename, url in models.items():
    file_path = os.path.join(model_dir, filename)
    if not os.path.exists(file_path):
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(url, file_path)
    else:
        print(f"Found {filename}.")


Found embedding_model.onnx.
Found embedding_model.tflite.
Found melspectrogram.onnx.
Found melspectrogram.tflite.


In [17]:
F = openwakeword.utils.AudioFeatures(
    melspec_model_path=f"{model_dir}/melspectrogram.onnx",
    embedding_model_path=f"{model_dir}/embedding_model.onnx",
    inference_framework="onnx",
)

## ⌀ Negative Clips

Get negative example paths, filtering out clips that are too long or too short

 pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cpu

In [20]:
import openwakeword.data

In [22]:
negative_clips, negative_durations = openwakeword.data.filter_audio_paths(
    [
        "./data/fma_sample",
        "./data/fsd50k_sample",
        "./data/speech_commands_test",
    ],
    min_length_secs=1.0, # minimum clip length in seconds
    max_length_secs=60*30, # maximum clip length in seconds
    duration_method="header" # use the file header to calculate duration
)

print(f"{len(negative_clips)} negative clips after filtering, representing ~{sum(negative_durations)//3600} hours")

200it [00:00, 5687.62it/s]
100%|██████████| 200/200 [00:01<00:00, 112.25it/s]
1000it [00:00, 13914.96it/s]
100%|██████████| 1000/1000 [00:03<00:00, 261.22it/s]
4890it [00:00, 19923.89it/s]
100%|██████████| 4890/4890 [00:07<00:00, 669.77it/s]

5987 negative clips after filtering, representing ~5.0 hours


Use HuggingFace datasets to load files from disk by batches, because:
- Load lazy
- Memmory-Mapping for doesn't load entire in RAM
- map() -> feature extraction, augmentation
- filter() -> remove audio too long/short

In [25]:
audio_dataset = datasets.Dataset.from_dict({
    "audio": negative_clips
})

In [26]:
audio_dataset = audio_dataset.cast_column(
    "audio",
    datasets.Audio(sampling_rate=16000)
)

- Get Audio Embeddings (features) for negative clips and save to .npy file
- Process files by batch and save to Numpy memory mapped file so that
- An array larger than the available system memory can be created

In [27]:
batch_size = 64 # number of files to load, compute features, and write to mmap at a time

In [28]:
clip_size = 3 # The desired window size (in seconds) for the trained openWakeWord model

In [31]:
N_total = int(sum(negative_durations)//clip_size) # Maximum number of rows in mmap file

N_total

6441

In [30]:
n_feature_cols = F.get_embedding_shape(clip_size)

n_feature_cols

(28, 96)

In [32]:
output_file = "negative_features.npy"

In [33]:
output_array_shape = (N_total, n_feature_cols[0], n_feature_cols[1])

output_array_shape

(6441, 28, 96)

In [34]:
fp = open_memmap(
    output_file,
    mode="w+",
    dtype=np.float32,
    shape=output_array_shape
)

fp

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [35]:
row_counter = 0

In [38]:
for i in tqdm(np.arange(0, audio_dataset.num_rows, batch_size)):
    # Load data in batches and shape into rectangular array
    wav_data = [
        (j["array"] * 32767).astype(np.int16) for j in audio_dataset[i:i + batch_size]["audio"]
    ]

    wav_data = openwakeword.data.stack_clips(
        wav_data,
        clip_size=16000*clip_size
    ).astype(np.int16)

    # Compute features (increase ncpu argument for faster processing)
    features = F.embed_clips(
        x=wav_data,
        batch_size=1024,
        ncpu=8
    )

    # Save computed features to mmap array file (stopping once the desired size is reached)
    if row_counter + features.shape[0] > N_total:
        fp[
            row_counter:min(row_counter+features.shape[0], N_total),
            :,
            :
        ] = features[
            0:N_total - row_counter,
            :,
            :
        ]

        fp.flush()
        break

    else:
        fp[
            row_counter:row_counter+features.shape[0],
            :,
            :
        ] = features

        row_counter += features.shape[0]
        fp.flush()

  0%|          | 0/94 [00:07<?, ?it/s]


In [39]:
openwakeword.data.trim_mmap(output_file)

Trimming empty rows: 7it [00:00,  8.58it/s]                       
